## Marmo Data Preparation

In [130]:
import pandas as pd
from datetime import datetime, timedelta
import holidays
import requests

## 1. Load the Data

In [146]:
# Read the sales table
sales201920 = pd.read_csv("C:/Users/ccui/Box Sync/SO Delivery_Marmo/Marmo_OPEX2020/Planning Marmo/2 Working Documents/data/Data Voor El 61/subset201920.csv",
                          sep=';',
                          header = 'infer',
                          dtype={"date": str, 
                                 "artikelnr": str,
                                 "besteldvolume": float,
                                 "klant": str,
                                 "branche": str}
                         )

sales20171819 = pd.read_csv("C:/Users/ccui/Box Sync/SO Delivery_Marmo/Marmo_OPEX2020/Planning Marmo/2 Working Documents/data/Data Voor El 61/subset20171819.csv",
                          sep=';',
                          header = 'infer',
                          dtype={"date": str, 
                                 "artikelnr": str,
                                 "besteld volume": float,
                                 "klant": str,
                                 "branche": str}
                         )

sales = sales20171819.append(sales201920, ignore_index=True)
sales = sales.dropna(subset=["date", "artikelnr", "besteldvolume", "klant", "branche"])

# parse the date column
dtm = lambda x: datetime.strptime(str(x), "%d/%m/%y")
sales["date"] = sales["date"].apply(dtm)

# cutoff the last two characters of the artikelnr 
sales['artikelnr'] = sales['artikelnr'].str[:11]

sales.head()

,date,artikelnr,besteldvolume,klant,branche
194937,2017-11-27,11001000320,3.84,6403,LEADER PRICE STEF
194938,2017-11-27,11001000320,3.84,6405,LEADER PRICE STEF
194939,2017-11-27,11001000320,4.80,6406,LEADER PRICE STEF
194940,2017-11-27,11001000320,3.84,6407,LEADER PRICE STEF
194941,2017-11-27,11001000320,5.76,6408,LEADER PRICE STEF


In [149]:
# aggregate per new artikelnr
sales = sales.groupby(['artikelnr', 'date', 'klant'], as_index=False).agg({'besteldvolume' : 'sum', 'branche': 'first'})
sales.head()

,artikelnr,date,klant,besteldvolume,branche
0,11001000000,2018-10-26,60344,0.64,LAMBRECHTS CENTRALE TONGEREN
1,11001000000,2018-10-26,60381,0.64,LAMBRECHTS CENTRALE TONGEREN
2,11001000000,2018-10-26,60391,1.28,LAMBRECHTS CENTRALE TONGEREN
3,11001000000,2018-10-26,60432,0.64,LAMBRECHTS CENTRALE TONGEREN
4,11001000000,2018-10-26,60449,0.64,LAMBRECHTS CENTRALE TONGEREN


In [153]:
# Check whether the volume column is float 
sales["besteldvolume"].sum(skipna=None)

9941893.55

In [154]:
# Read the weather table
weather = pd.read_csv("C:/Users/ccui/Box Sync/Charles/Marmo/data/weatherHistoryDarksky20172020.csv",
                          sep=';',
                          header = 'infer'
                         )

# parse the date column
dtm = lambda x: datetime.strptime(str(x), "%Y-%m-%d")
weather["date"] = weather["date"].apply(dtm)
weather = weather.drop("DataType", axis=1)
weather.head()

,date,Wind,Humidity,CloudCover,TemperatureMin,TemperatureMax,PrecipProbability,Icon
0,2017-01-01,6.59,0.94,0.81,-3.300000,0.188889,0.18,partly-cloudy-day
1,2017-01-02,3.62,0.96,0.57,-0.711111,1.144444,0.53,snow
2,2017-01-03,7.39,0.95,0.52,-1.744444,2.711111,0.18,partly-cloudy-day
3,2017-01-04,8.86,0.89,0.65,2.111111,7.177778,0.97,rain
4,2017-01-05,5.09,0.83,0.28,-2.583333,4.650000,0.27,rain


In [155]:
# Read the promo data
promo = pd.read_csv("C:/Users/ccui/Box Sync/SO Delivery_Marmo/Marmo_OPEX2020/Planning Marmo/2 Working Documents/data/Data Voor El 61/promoTable.csv",
                    sep=';',
                    header = 'infer',
                    dtype = {"date": str, 
                             "promoType": str,
                             "artikelnr": str}
                         )

# parse the date column
dtm = lambda x: datetime.strptime(str(x), "%Y-%m-%d")
promo["date"] = promo["date"].apply(dtm)

# make the promotype all small letters to avoid Folder & folder being different 
promo['promoType'] = promo['promoType'].str.lower()

# cutoff the last two characters of the artikelnr 
promo['artikelnr'] = promo['artikelnr'].str[:11]

promo.head()

,date,promoType,artikelnr
0,2017-01-02,folder,11304000010
1,2017-01-03,folder,11304000010
2,2017-01-04,folder,11304000010
3,2017-01-05,folder,11304000010
4,2017-01-06,folder,11304000010


## 2. Add the calender features

## 5. Handle the categorical features

In [161]:
# select the categorical variables
cat_features = baseTable.columns.drop(['date', 'artikelnr', 'besteldvolume', 'klant', 'year', 
                                   'Wind', 'Humidity', 'CloudCover', 'TemperatureMin', 'TemperatureMax', 'PrecipProbability'])
print(cat_features)

Index(['branche', 'month', 'week', 'day', 'weekday', 'day_name', 'holiday',
       'after_holiday', 'before_holiday', 'bridge_day', 'long_weekend',
       'before_long_weekend', 'after_long_weekend', 'weekend',
       'school_vacation', 'krokus', 'pasen', 'zomer', 'herfst', 'kerst',
       'week_after_school_vacation', 'week_after_krokus', 'week_after_pasen',
       'week_after_zomer', 'week_after_herfst', 'week_after_kerst',
       'week_before_school_vacation', 'week_before_krokus',
       'week_before_pasen', 'week_before_zomer', 'week_before_herfst',
       'week_before_kerst', 'Icon', 'promoType'],
      dtype='object')


In [162]:
# set variables to categorical type
baseTable[cat_features] = baseTable[cat_features].astype('category')

# turn the categorical variables to integers to enable the lgb algorithm to handle it
baseTable[cat_features] = baseTable[cat_features].apply(lambda x: x.cat.codes)

baseTable.head()

,artikelnr,date,klant,besteldvolume,branche,year,month,week,day,weekday,...,week_before_herfst,week_before_kerst,Wind,Humidity,CloudCover,TemperatureMin,TemperatureMax,PrecipProbability,Icon,promoType
0,11001000000,2018-10-26,60344,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
1,11001000000,2018-10-26,60381,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
2,11001000000,2018-10-26,60391,1.28,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
3,11001000000,2018-10-26,60432,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1
4,11001000000,2018-10-26,60449,0.64,37,2018,9,42,25,4,...,0,0,9.67,0.77,0.87,6.644444,12.561111,0.0,0,-1


In [163]:
# check whether all the X features are float or int
baseTable.dtypes

artikelnr                              object
date                           datetime64[ns]
klant                                  object
besteldvolume                         float64
branche                                  int8
year                                    int64
month                                    int8
week                                     int8
day                                      int8
weekday                                  int8
day_name                                 int8
holiday                                  int8
after_holiday                            int8
before_holiday                           int8
bridge_day                               int8
long_weekend                             int8
before_long_weekend                      int8
after_long_weekend                       int8
weekend                                  int8
school_vacation                          int8
krokus                                   int8
pasen                             

## 5. Write the prepared data to csv to use in the modeling script

In [164]:
# write the sales table to csv to read in the modeling script (only take 2019 & 2020)\n",
baseTableToModel = baseTable[(baseTable["date"]>=datetime(2017, 1, 1)) 
                             & (baseTable["date"]<=datetime(2020, 5, 31))]

baseTableToModel.to_csv("C:/Users/ccui/Box Sync/SO Delivery_Marmo/Marmo_OPEX2020/Planning Marmo/2 Working Documents/data/Data Voor El 61/salesToModel.csv", 
                        sep =';',
                        index = False)